### Main file

In [1]:
from tools.helpers import *
from tools.srbm import *
from tools.parameters import *
from brian2 import *
from brian2tools import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# ! deactivated the long taking part - needs reactivation !

Id = create_Id()
W, b_v, b_c, b_h = create_rbm_parameters()
mnist_data = load_mnist_data(min_p = .00001, max_p = .98, binary = True, seed=0)
loc = main(W, b_v, b_c, b_h, Id =create_Id(), monitors = True, mnist_data=mnist_data)
locals().update(loc)

Creating equation
Creating Population
